In [1]:
#To run:
#ipython simple_nn.py --pylab -i

import pandas as pd
import numpy as np
import datetime

import torch
from torch.autograd import Variable

In [2]:
#load data
# df_merged_volume = pd.read_pickle("phase1_training_vol_route_weather_joined_table.pkl")

df_merged_volume = pd.read_csv('../data/preprocessed_input_interpolate_20min.csv')

In [3]:
df_merged_volume

,"(1, 0, 'cargocar')","(1, 0, 'etc')","(1, 0, 'motorcycle')","(1, 0, 'privatecar')","(1, 0, 'tot')","(1, 0, 'unknowncar')","(1, 1, 'cargocar')","(1, 1, 'etc')","(1, 1, 'motorcycle')","(1, 1, 'privatecar')",...,hour,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation,dayofweek,is_holiday
0,0,1.0,0,0,14.0,14.0,38.0,25.0,89.0,12.0,...,0,1013.200000,1018.200000,341.000000,3.500000,21.100000,68.000000,0.0,1,0
1,0,0.0,0,0,13.0,13.0,24.0,11.0,41.0,15.0,...,0,1013.233333,1018.233333,342.444444,3.400000,21.411111,66.888889,0.0,1,0
2,0,1.0,0,0,7.0,7.0,10.0,7.0,22.0,5.0,...,0,1013.266667,1018.266667,343.888889,3.300000,21.722222,65.777778,0.0,1,0
3,0,2.0,0,0,6.0,6.0,3.0,0.0,3.0,0.0,...,1,1013.300000,1018.300000,345.333333,3.200000,22.033333,64.666667,0.0,1,0
4,0,1.0,0,0,5.0,5.0,5.0,0.0,3.0,0.0,...,1,1013.333333,1018.333333,346.777778,3.100000,22.344444,63.555556,0.0,1,0
5,0,1.0,0,0,5.0,5.0,3.0,1.0,8.0,0.0,...,1,1013.366667,1018.366667,348.222222,3.000000,22.655556,62.444444,0.0,1,0
6,0,0.0,0,0,6.0,6.0,6.0,1.0,2.0,0.0,...,2,1013.400000,1018.400000,349.666667,2.900000,22.966667,61.333333,0.0,1,0
7,0,2.0,0,0,9.0,9.0,8.0,1.0,4.0,0.0,...,2,1013.433333,1018.433333,351.111111,2.800000,23.277778,60.222222,0.0,1,0
8,0,0.0,0,0,7.0,7.0,4.0,1.0,4.0,0.0,...,2,1013.466667,1018.466667,352.555556,2.700000,23.588889,59.111111,0.0,1,0
9,0,0.0,0,0,10.0,10.0,2.0,0.0,0.0,0.0,...,3,1013.500000,1018.500000,354.000000,2.600000,23.900000,58.000000,0.0,1,0


In [4]:
for i in df_merged_volume.columns:
    print(i)

(1, 0, 'cargocar')
(1, 0, 'etc')
(1, 0, 'motorcycle')
(1, 0, 'privatecar')
(1, 0, 'tot')
(1, 0, 'unknowncar')
(1, 1, 'cargocar')
(1, 1, 'etc')
(1, 1, 'motorcycle')
(1, 1, 'privatecar')
(1, 1, 'tot')
(1, 1, 'unknowncar')
(2, 0, 'cargocar')
(2, 0, 'etc')
(2, 0, 'motorcycle')
(2, 0, 'privatecar')
(2, 0, 'tot')
(2, 0, 'unknowncar')
(3, 0, 'cargocar')
(3, 0, 'etc')
(3, 0, 'motorcycle')
(3, 0, 'privatecar')
(3, 0, 'tot')
(3, 0, 'unknowncar')
(3, 1, 'cargocar')
(3, 1, 'etc')
(3, 1, 'motorcycle')
(3, 1, 'privatecar')
(3, 1, 'tot')
(3, 1, 'unknowncar')
('A', 2)
('A', 3)
('B', 1)
('B', 3)
('C', 1)
('C', 3)
date
hour
pressure
sea_pressure
wind_direction
wind_speed
temperature
rel_humidity
precipitation
dayofweek
is_holiday


In [26]:
'''
5 Checkpoint: 1-0 , 1-1, 2-0, 3-0, 3-1
    2nd index{ 0: entry, 1: exit }
    
6 Paths: A-2, A-3, B-1, B-3, C-1, C-3


Features:
1. Volume (total) (20m)
-
2. ETC (20m)
3. Motorcycle (20m)
4. Cargocar (20m)
5. Privatecar (20m)
6. Unknowncar (20m)
-
7. routetime median (20m)
-
8. pressure (3 hrs)
9. sea_pressure (3 hrs)
10. wind_direction (3 hrs)
11. wind_speed (3 hrs)
12. temperature (3 hrs)
13. rel_humidity (3 hrs)
14. precipitation (3 hrs)

--
15. date
16. hour
17. dayofweek
18. is_holiday


== TODO:

1. Create the Input Tensor (3D)

== Done:
1. Investigate Backfill for FillNA: Solved by interpolate (by Panda function)
2. Moving avg for 3hr's features (interpolate)


== Datasets:
Trajectories [2016,7,19] - [2016,10,17]
Volume       [2016,9,19] - [2016,10,17]
Weather      [2016,7, 1] - [2016,10,17]



== Procedures:
Preprocessing
0. 
1. Backfill for FillNA
2. Moving avg for 3hr's features
3. Select the relevant row (time duration, 6am and 3pm)
4. Normalization
5. Create the Input Tensor (3D)

Build NN:
Keras
'''

for i in df_merged_volume.columns:
    print(i)

(1, 0, 'cargocar')
(1, 0, 'etc')
(1, 0, 'motorcycle')
(1, 0, 'privatecar')
(1, 0, 'tot')
(1, 0, 'unknowncar')
(1, 1, 'cargocar')
(1, 1, 'etc')
(1, 1, 'motorcycle')
(1, 1, 'privatecar')
(1, 1, 'tot')
(1, 1, 'unknowncar')
(2, 0, 'cargocar')
(2, 0, 'etc')
(2, 0, 'motorcycle')
(2, 0, 'privatecar')
(2, 0, 'tot')
(2, 0, 'unknowncar')
(3, 0, 'cargocar')
(3, 0, 'etc')
(3, 0, 'motorcycle')
(3, 0, 'privatecar')
(3, 0, 'tot')
(3, 0, 'unknowncar')
(3, 1, 'cargocar')
(3, 1, 'etc')
(3, 1, 'motorcycle')
(3, 1, 'privatecar')
(3, 1, 'tot')
(3, 1, 'unknowncar')
('A', 2)
('A', 3)
('B', 1)
('B', 3)
('C', 1)
('C', 3)
date
hour
pressure
sea_pressure
wind_direction
wind_speed
temperature
rel_humidity
precipitation
dayofweek
is_holiday


In [23]:
# only select relevant time, 6am (4 hrs info)
lower_sel_time = 6
upper_sel_time = 9

sel_rows = df_merged_volume[ lambda df : (df.hour>=lower_sel_time) & (df.hour<=upper_sel_time)]

In [24]:
sel_rows

,"(1, 0, 'cargocar')","(1, 0, 'etc')","(1, 0, 'motorcycle')","(1, 0, 'privatecar')","(1, 0, 'tot')","(1, 0, 'unknowncar')","(1, 1, 'cargocar')","(1, 1, 'etc')","(1, 1, 'motorcycle')","(1, 1, 'privatecar')",...,hour,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation,dayofweek,is_holiday
18,0,4.0,0,0,13.0,13.0,23.0,4.0,24.0,0.0,...,6,1012.200000,1017.200000,7.000000,4.600000,26.900000,43.000000,0.000000,1,0
19,0,4.0,0,0,20.0,20.0,24.0,3.0,35.0,0.0,...,6,1012.244444,1017.244444,7.444444,4.588889,26.733333,42.888889,0.000000,1,0
20,0,8.0,0,0,29.0,29.0,28.0,9.0,40.0,1.0,...,6,1012.288889,1017.288889,7.888889,4.577778,26.566667,42.777778,0.000000,1,0
21,0,14.0,0,0,38.0,38.0,20.0,23.0,65.0,3.0,...,7,1012.333333,1017.333333,8.333333,4.566667,26.400000,42.666667,0.000000,1,0
22,0,7.0,0,0,30.0,30.0,26.0,32.0,96.0,4.0,...,7,1012.377778,1017.377778,8.777778,4.555556,26.233333,42.555556,0.000000,1,0
23,0,9.0,0,0,31.0,31.0,20.0,40.0,117.0,2.0,...,7,1012.422222,1017.422222,9.222222,4.544444,26.066667,42.444444,0.000000,1,0
24,0,21.0,0,0,53.0,53.0,24.0,37.0,100.0,8.0,...,8,1012.466667,1017.466667,9.666667,4.533333,25.900000,42.333333,0.000000,1,0
25,0,21.0,0,0,50.0,50.0,19.0,32.0,127.0,5.0,...,8,1012.511111,1017.511111,10.111111,4.522222,25.733333,42.222222,0.000000,1,0
26,0,15.0,0,0,45.0,45.0,22.0,23.0,108.0,1.0,...,8,1012.555556,1017.555556,10.555556,4.511111,25.566667,42.111111,0.000000,1,0
27,0,13.0,0,0,51.0,51.0,28.0,29.0,115.0,7.0,...,9,1012.600000,1017.600000,11.000000,4.500000,25.400000,42.000000,0.000000,1,0


In [8]:
max_len = 6  # Sliding Window size



#replace NA entries and normalize data
sel_rows_mean = sel_rows.mean(0)
sel_rows_std = sel_rows.std(0)

In [10]:
sel_rows_norm = sel_rows.fillna(sel_rows_mean)
sel_rows_norm = (sel_rows_norm - sel_rows_mean) / sel_rows_std

/Users/Kin/anaconda/lib/python3.5/site-packages/pandas/core/ops.py:995: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1 but corresponding boolean dimension is 56
  yrav = yrav[mask]


In [11]:
len(sel_rows_norm)

56

In [ ]:
#split data to train and test set
np.random.seed(1234)
mask = np.random.rand(len(sel_rows_norm)) < 0.75
train_rows = sel_rows_norm[mask]
test_rows = sel_rows_norm[~mask]

#select useful feature input cols and output cols
vols = [(1,0), (1,1), (2,0), (3,0), (3,1)]
routes = [ ("A",2), ("A",3), ("B",1), ("B",3), ("C",1), ("C",3) ]

inCols = []
for dt in range(0,120,20):
  for (tid,io) in vols:
    inCols.append('dt%i_%s%s_%s_vol' %(dt,tid,io,'total') )
  for (p,q) in routes:
    inCols.append('dt%i_%s%s_routetime_median'%(dt,p,q) )

outCols = []
for dt in range(120,240,20):
  for (tid,io) in vols:
    outCols.append('dt%i_%s%s_%s_vol' %(dt,tid,io,'total') )
  for (p,q) in routes:
    outCols.append('dt%i_%s%s_routetime_median'%(dt,p,q) )

train_input = train_rows[inCols]
train_ans = train_rows[outCols]

test_input = test_rows[inCols]
test_ans = test_rows[outCols]

inDim = len(inCols)
outDim = len(outCols)

#-----------------------------------
#pytorch part
#-----------------------------------
# Create random Tensors to hold inputs and outputs, and wrap them in Variables.
x = Variable(torch.FloatTensor(train_input.as_matrix()), requires_grad=False)
y = Variable(torch.FloatTensor(train_ans.as_matrix()  ), requires_grad=False)

x_test = Variable(torch.FloatTensor(test_input.as_matrix()), requires_grad=False)
y_test = Variable(torch.FloatTensor(test_ans.as_matrix()  ), requires_grad=False)

# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(inDim, 10),
    torch.nn.ReLU(),
    torch.nn.Linear(10, 10),
    torch.nn.ReLU(),
    torch.nn.Linear(10, outDim),
)
loss_fn = torch.nn.MSELoss(size_average=False)

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Variables it should update.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

loss_seq = []
loss_test_seq = []

for t in range(3000):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x)
    y_pred_test = model(x_test)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    loss_test = loss_fn(y_pred_test, y_test)
    print(t, loss.data[0], loss_test.data[0])

    loss_seq.append( loss.data[0] )
    loss_test_seq.append( loss_test.data[0] )

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable weights
    # of the model)
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

plot(loss_seq)
plot(loss_test_seq)

print min(loss_test_seq) , loss_test_seq.index(min(loss_test_seq))